# 🧪 DataHub Glossary를 온톨로지로 쓸 수 있을까 — 실습

> **DataNexus Building Log #003** 실습 노트북
>
> 📝 블로그 원문: [DataHub Glossary를 온톨로지로 쓸 수 있을까](https://biz-agentic-ai.github.io/posts/datanexus/003-datahub-glossary-as-ontology/)

---

## 이 노트북에서 다루는 것

1. **NetworkX로 온톨로지 그래프 구축** — DozerDB Cypher 쿼리를 Python으로 재현
2. **관계 유형 분리** — `RelatedTo` 하나로 퉁치는 것 vs. `MANUFACTURES`/`STOCKS` 분리의 차이
3. **엣지 속성 부여** — confidence, temporal, cardinality
4. **파생 지표 그래프** — `CALCULATED_FROM` 관계로 계산식 표현
5. **시각화** — 온톨로지 그래프를 눈으로 확인

> ⚠️ 실제 DozerDB/Neo4j 없이도 실행 가능합니다. NetworkX로 그래프 DB 개념을 시뮬레이션합니다.

## 0. 환경 설정

In [ ]:
# Colab 기본 라이브러리로 충분합니다
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Colab 환경)
!apt-get -qq install -y fonts-nanum > /dev/null 2>&1
fe = fm.FontEntry(fname='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumGothic')
fm.fontManager.ttflist.append(fe)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

print('✅ 환경 설정 완료!')

---
## 1. 문제 재현: `RelatedTo` 하나로 퉁치면 생기는 일

블로그에서 다뤘던 핵심 문제를 코드로 재현합니다.

> "A 공장에서 **생산된** B 제품"과 "A 공장에 **재고가 있는** B 제품". DataHub Glossary에서는 둘 다 `RelatedTo`.

In [ ]:
# --- DataHub 방식: RelatedTo 하나로 표현 ---
datahub_graph = nx.MultiDiGraph()  # 동일 노드 간 여러 엣지 허용

# 용어 등록
datahub_graph.add_node('A공장', type='Factory', definition='서울 소재 공장')
datahub_graph.add_node('B제품', type='Product', definition='주력 제품')

# DataHub Glossary에서는 관계 유형이 4개뿐
# 생산과 재고 모두 RelatedTo로 표현할 수밖에 없다
datahub_graph.add_edge('A공장', 'B제품', relation='RelatedTo')  # 생산? 재고?
datahub_graph.add_edge('A공장', 'B제품', relation='RelatedTo')  # 생산? 재고?

print('=== DataHub Glossary 방식 ===')
print(f'노드: {list(datahub_graph.nodes(data=True))}')
print(f'엣지: {list(datahub_graph.edges(data=True))}')
print()

# 시뮬레이션: "A 공장에서 생산된 제품은?" 질문 처리
print('💬 사용자 질문: "A 공장에서 생산된 제품은?"')
print()
edges = list(datahub_graph.edges(data=True))
related = [e for e in edges if e[0] == 'A공장' and e[2]['relation'] == 'RelatedTo']
print(f'🔍 온톨로지 조회: A공장 → RelatedTo → {[e[1] for e in related]}')
print(f'   관계 {len(related)}개 발견, 모두 "RelatedTo"')
print(f'   → ❌ 어느 것이 "생산"이고 어느 것이 "재고"인지 구분 불가!')
print(f'   → LLM이 production 대신 inventory 테이블을 JOIN할 수 있음')

---
## 2. 해결: DozerDB 방식 — 세분화된 관계 유형

DozerDB(그래프 DB)에서는 관계 유형을 자유롭게 정의합니다.

아래 코드는 블로그의 Cypher 쿼리를 Python(NetworkX)으로 1:1 변환한 것입니다.

```
Cypher:
  CREATE (factory)-[:MANUFACTURES {...}]->(product)
  CREATE (factory)-[:STOCKS {...}]->(product)

Python (NetworkX):
  G.add_edge('A공장', 'B제품', key='MANUFACTURES', ...)
  G.add_edge('A공장', 'B제품', key='STOCKS', ...)
```

In [ ]:
# --- DozerDB 방식: 세분화된 관계 유형 + 엣지 속성 ---
dozerdb_graph = nx.MultiDiGraph()

# 엔티티 생성 (DataHub에서 동기화된 용어)
dozerdb_graph.add_node('A공장', type='Factory', definition='서울 소재 공장')
dozerdb_graph.add_node('B제품', type='Product', definition='주력 제품')

# 생산 관계 — 시작 시점과 신뢰도를 속성으로 기록
dozerdb_graph.add_edge('A공장', 'B제품',
    key='MANUFACTURES',
    since='2024-01-01',
    confidence=0.95
)

# 재고 관계 — 별도 엣지, 수량과 갱신 시점
dozerdb_graph.add_edge('A공장', 'B제품',
    key='STOCKS',
    quantity=500,
    last_updated='2026-02-01'
)

print('=== DozerDB 방식 ===')
for u, v, key, data in dozerdb_graph.edges(keys=True, data=True):
    print(f'  {u} —[{key}]→ {v}')
    for k, val in data.items():
        print(f'      {k}: {val}')
    print()

In [ ]:
# 시뮬레이션: 같은 질문, 다른 결과
print('💬 사용자 질문: "A 공장에서 생산된 제품은?"')
print()

# DozerDB에서는 관계 유형으로 정확히 필터링
manufactures = [
    (u, v, data)
    for u, v, key, data in dozerdb_graph.edges(keys=True, data=True)
    if u == 'A공장' and key == 'MANUFACTURES'
]

stocks = [
    (u, v, data)
    for u, v, key, data in dozerdb_graph.edges(keys=True, data=True)
    if u == 'A공장' and key == 'STOCKS'
]

print(f'🔍 온톨로지 조회: A공장 → MANUFACTURES → {[e[1] for e in manufactures]}')
print(f'   confidence: {manufactures[0][2]["confidence"]}')
print(f'   → ✅ production 테이블 JOIN (정확한 결과)')
print()
print(f'📦 참고: A공장 → STOCKS → {[e[1] for e in stocks]}')
print(f'   quantity: {stocks[0][2]["quantity"]}')
print(f'   → inventory 테이블은 이 질문과 무관 → 자동 제외')

---
## 3. 엣지 속성이 왜 중요한가 — 신뢰도, 유효 기간

블로그에서 언급한 핵심 한계:
> DataHub Glossary에서 "A RelatedTo B"를 설정하면, 그 관계에 아무것도 더 붙일 수 없다.

In [ ]:
# 실무 시나리오: 조직 개편으로 부서-제품 매핑이 변경된 경우
G = nx.MultiDiGraph()

G.add_node('영업1팀', type='Department')
G.add_node('영업2팀', type='Department')
G.add_node('프리미엄라인', type='ProductLine')

# 과거 관계: 영업1팀이 프리미엄라인 담당 (2024년까지)
G.add_edge('영업1팀', '프리미엄라인',
    key='MANAGES',
    valid_from='2023-01-01',
    valid_to='2024-12-31',   # ← 만료됨
    confidence=0.95,
    source='HR시스템'
)

# 현재 관계: 조직 개편 후 영업2팀으로 이관
G.add_edge('영업2팀', '프리미엄라인',
    key='MANAGES',
    valid_from='2025-01-01',
    valid_to=None,           # ← 현재 유효
    confidence=0.95,
    source='HR시스템'
)

print('=== 시간축이 있는 관계 (Temporal Relationships) ===')
print()

from datetime import datetime
query_date = '2026-02-18'

print(f'📅 조회 시점: {query_date}')
print()

for u, v, key, data in G.edges(keys=True, data=True):
    valid_to = data.get('valid_to')
    is_current = valid_to is None or valid_to >= query_date
    status = '✅ 현재 유효' if is_current else '⛔ 만료됨'
    print(f'  {u} —[{key}]→ {v}  [{status}]')
    print(f'      유효기간: {data["valid_from"]} ~ {valid_to or "현재"}')
    print(f'      confidence: {data["confidence"]}, source: {data["source"]}')
    print()

# 현재 유효한 관계만 필터링
current_manager = [
    (u, v) for u, v, key, data in G.edges(keys=True, data=True)
    if key == 'MANAGES'
    and (data.get('valid_to') is None or data['valid_to'] >= query_date)
]
print(f'→ 현재 프리미엄라인 담당: {current_manager[0][0]}')
print(f'→ DataHub Glossary에서는 이런 시간축 관리가 불가능')

---
## 4. 파생 지표를 그래프로 표현: `CALCULATED_FROM`

> "순매출 = 총매출 - 반품 - 에누리"를 Excel이 아닌 그래프 관계로 관리

블로그의 Cypher 쿼리를 그대로 재현합니다.

In [ ]:
# 파생 지표 그래프 구축
metrics = nx.MultiDiGraph()

# 엔티티 (용어) 등록
terms = {
    '순매출': {'type': 'DerivedMetric', 'definition': '총매출에서 반품과 에누리를 뺀 금액'},
    '총매출': {'type': 'BaseMetric',    'definition': '할인 전 전체 매출액'},
    '반품':   {'type': 'BaseMetric',    'definition': '고객 반품에 의한 차감액'},
    '에누리': {'type': 'BaseMetric',    'definition': '할인 및 에누리 차감액'},
}

for name, attrs in terms.items():
    metrics.add_node(name, **attrs)

# 계산 관계: 순매출 = 총매출 - 반품 - 에누리
# Cypher: CREATE (net)-[:CALCULATED_FROM {operator: 'subtract'}]->(gross)
metrics.add_edge('순매출', '총매출', key='CALCULATED_FROM', operator='base',     order=1)
metrics.add_edge('순매출', '반품',   key='CALCULATED_FROM', operator='subtract', order=2)
metrics.add_edge('순매출', '에누리', key='CALCULATED_FROM', operator='subtract', order=3)

print('=== 파생 지표 계산 그래프 ===')
print()

# 순매출의 계산식을 그래프에서 자동 추출
calc_edges = sorted(
    [(v, data) for u, v, key, data in metrics.edges(keys=True, data=True)
     if u == '순매출' and key == 'CALCULATED_FROM'],
    key=lambda x: x[1]['order']
)

formula_parts = []
for term_name, data in calc_edges:
    op = data['operator']
    if op == 'base':
        formula_parts.append(term_name)
    elif op == 'subtract':
        formula_parts.append(f'- {term_name}')
    elif op == 'add':
        formula_parts.append(f'+ {term_name}')

print(f'📊 순매출 = {" ".join(formula_parts)}')
print()
print('그래프에서 계산식이 자동 추출됩니다.')
print('Excel 시트에 묻혀있는 것보다 추적과 변경 관리가 쉽습니다.')

In [ ]:
# 변경 시뮬레이션: "할인"이 추가되면?
print('=== 변경 시뮬레이션: 계산식에 "할인" 항목 추가 ===')
print()

metrics.add_node('할인', type='BaseMetric', definition='프로모션 할인 차감액')
metrics.add_edge('순매출', '할인', key='CALCULATED_FROM', operator='subtract', order=4)

# 변경 후 계산식 자동 재추출
calc_edges_updated = sorted(
    [(v, data) for u, v, key, data in metrics.edges(keys=True, data=True)
     if u == '순매출' and key == 'CALCULATED_FROM'],
    key=lambda x: x[1]['order']
)

formula_parts_updated = []
for term_name, data in calc_edges_updated:
    op = data['operator']
    if op == 'base':
        formula_parts_updated.append(term_name)
    elif op == 'subtract':
        formula_parts_updated.append(f'- {term_name}')

print(f'📊 순매출(변경 후) = {" ".join(formula_parts_updated)}')
print()
print('→ 관계 하나만 추가하면 계산식이 자동 반영됩니다.')
print('→ Excel처럼 "어딘가에 적혀 있는 걸 찾아서 수정"할 필요 없음')

---
## 5. 시각화: 전체 온톨로지 그래프

> DataHub UI는 Lineage(데이터 계보) 탐색에 맞춰져 있어, 다대다 그래프를 탐색하는 화면 자체가 없다.

그래프 DB에서는 시각화가 자유롭습니다.

In [ ]:
# 통합 온톨로지 그래프 구축
ontology = nx.MultiDiGraph()

# --- 엔티티 등록 ---
entities = {
    'A공장':      {'type': 'Factory',       'color': '#e74c3c'},
    'B제품':      {'type': 'Product',       'color': '#3498db'},
    '순매출':     {'type': 'DerivedMetric', 'color': '#2ecc71'},
    '총매출':     {'type': 'BaseMetric',    'color': '#27ae60'},
    '반품':       {'type': 'BaseMetric',    'color': '#27ae60'},
    '에누리':     {'type': 'BaseMetric',    'color': '#27ae60'},
    '영업2팀':    {'type': 'Department',    'color': '#9b59b6'},
    '프리미엄라인': {'type': 'ProductLine',  'color': '#f39c12'},
}

for name, attrs in entities.items():
    ontology.add_node(name, **attrs)

# --- 관계 등록 ---
ontology.add_edge('A공장', 'B제품', key='MANUFACTURES', color='#e67e22')
ontology.add_edge('A공장', 'B제품', key='STOCKS',       color='#e67e22')
ontology.add_edge('B제품', '프리미엄라인', key='BELONGS_TO', color='#1abc9c')
ontology.add_edge('영업2팀', '프리미엄라인', key='MANAGES',  color='#8e44ad')
ontology.add_edge('순매출', '총매출', key='CALCULATED_FROM', color='#2ecc71')
ontology.add_edge('순매출', '반품', key='CALCULATED_FROM',   color='#2ecc71')
ontology.add_edge('순매출', '에누리', key='CALCULATED_FROM', color='#2ecc71')

# --- 시각화 ---
fig, ax = plt.subplots(1, 1, figsize=(14, 9))
ax.set_facecolor('#1a1a2e')
fig.patch.set_facecolor('#1a1a2e')

pos = nx.spring_layout(ontology, k=2.5, seed=42)

# 노드 그리기
node_colors = [entities[n]['color'] for n in ontology.nodes()]
nx.draw_networkx_nodes(ontology, pos, ax=ax,
    node_color=node_colors, node_size=2000, alpha=0.9,
    edgecolors='white', linewidths=2
)
nx.draw_networkx_labels(ontology, pos, ax=ax,
    font_size=10, font_color='white', font_weight='bold'
)

# 엣지 그리기 (관계 유형별 색상)
for u, v, key, data in ontology.edges(keys=True, data=True):
    nx.draw_networkx_edges(ontology, pos, ax=ax,
        edgelist=[(u, v)],
        edge_color=data.get('color', '#ffffff'),
        width=2, alpha=0.7,
        arrows=True, arrowsize=20,
        connectionstyle=f'arc3,rad=0.1'
    )

# 엣지 라벨 (관계 유형)
edge_labels = {}
for u, v, key, data in ontology.edges(keys=True, data=True):
    label_key = (u, v)
    if label_key in edge_labels:
        edge_labels[label_key] += f'\n{key}'
    else:
        edge_labels[label_key] = key

nx.draw_networkx_edge_labels(ontology, pos, ax=ax,
    edge_labels=edge_labels,
    font_size=8, font_color='#ecf0f1',
    bbox=dict(boxstyle='round,pad=0.3', facecolor='#2c3e50', alpha=0.8)
)

# 범례
legend_items = {
    'Factory': '#e74c3c',
    'Product': '#3498db',
    'DerivedMetric': '#2ecc71',
    'BaseMetric': '#27ae60',
    'Department': '#9b59b6',
    'ProductLine': '#f39c12',
}
for label, color in legend_items.items():
    ax.scatter([], [], c=color, s=100, label=label)
ax.legend(loc='upper left', fontsize=9, facecolor='#16213e',
          edgecolor='#ecf0f1', labelcolor='white')

ax.set_title('DataNexus 온톨로지 그래프 (DozerDB 시뮬레이션)',
             color='white', fontsize=14, fontweight='bold', pad=20)
ax.axis('off')
plt.tight_layout()
plt.show()

print('\n💡 DataHub UI에서는 이런 다대다 그래프 탐색이 불가능합니다.')
print('   그래프 DB를 사용하면 온톨로지 전체를 시각적으로 탐색할 수 있습니다.')

---
## 6. 비교 정리: DataHub Glossary vs. DozerDB

블로그의 "되는 것과 안 되는 것" 표를 코드로 검증합니다.

In [ ]:
# 비교표를 HTML로 출력
comparison_html = """
<style>
  .comparison-table { border-collapse: collapse; width: 100%; font-family: sans-serif; }
  .comparison-table th, .comparison-table td { padding: 12px 16px; text-align: left; border: 1px solid #34495e; }
  .comparison-table th { background-color: #2c3e50; color: white; font-weight: bold; }
  .comparison-table tr:nth-child(even) { background-color: #ecf0f1; }
  .comparison-table tr:nth-child(odd) { background-color: #ffffff; }
  .check { color: #27ae60; font-weight: bold; }
  .cross { color: #e74c3c; font-weight: bold; }
</style>
<table class="comparison-table">
  <tr>
    <th>기능</th>
    <th>DataHub Glossary</th>
    <th>DozerDB</th>
    <th>이 노트북에서 검증</th>
  </tr>
  <tr>
    <td>용어 정의 (name, definition)</td>
    <td class="check">✅ 가능</td>
    <td class="check">✅ 가능</td>
    <td>섹션 1, 2</td>
  </tr>
  <tr>
    <td>세분화된 관계 유형</td>
    <td class="cross">❌ 4종만 (IsA, HasA, RelatedTo, Values)</td>
    <td class="check">✅ 무제한 (MANUFACTURES, STOCKS 등)</td>
    <td>섹션 1 → 2</td>
  </tr>
  <tr>
    <td>엣지 속성 (confidence, temporal)</td>
    <td class="cross">❌ 불가</td>
    <td class="check">✅ 자유롭게 부여</td>
    <td>섹션 3</td>
  </tr>
  <tr>
    <td>파생 지표 계산식</td>
    <td class="cross">❌ 표현 불가</td>
    <td class="check">✅ CALCULATED_FROM 관계</td>
    <td>섹션 4</td>
  </tr>
  <tr>
    <td>그래프 시각화</td>
    <td class="cross">❌ Lineage만 가능</td>
    <td class="check">✅ 다대다 그래프 탐색</td>
    <td>섹션 5</td>
  </tr>
  <tr>
    <td>관계 유형 확장 시</td>
    <td class="cross">❌ PDL 수정 → 재배포</td>
    <td class="check">✅ CREATE 쿼리 한 줄</td>
    <td>섹션 2</td>
  </tr>
</table>
"""

display(HTML(comparison_html))

---
## 🎯 핵심 정리

1. **DataHub Glossary**는 용어 사전으로 훌륭하지만, 온톨로지 저장소로는 표현력이 부족
2. **관계 유형이 4개뿐**이라 비즈니스 맥락을 담을 수 없음 → NL2SQL 정확도에 직접 영향
3. **DozerDB**(그래프 DB)를 도입해 역할 분리
   - DataHub: 용어 정의의 Source of Truth
   - DozerDB: 세분화된 관계, 엣지 속성, 그래프 탐색
4. 파생 지표도 그래프 관계(`CALCULATED_FROM`)로 관리 → 변경 추적 용이

---

📖 **다음 글**: SKOS 호환 레이어를 왜 넣었는지 (표준 온톨로지 포맷 지원)

🔗 **DataNexus 시리즈**: [블로그](https://biz-agentic-ai.github.io/) | [GitHub](https://github.com/biz-agentic-ai)